In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/projects/fc-gaga'
LOGDIR = "logs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/projects/fc-gaga


In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
print("Tensorflow version:", tf.__version__)


from dataset import Dataset
from model import Trainer, Parameters
from model import hyperparams_defaults as hyperparams_dict

Tensorflow version: 2.2.0


In [0]:
hyperparams_dict["dataset"] = 'metr-la'
hyperparams_dict["horizon"] = 12
hyperparams_dict["history_length"] = 12

dataset = Dataset(name=hyperparams_dict["dataset"], horizon=hyperparams_dict["horizon"], history_length=hyperparams_dict["history_length"])

hyperparams_dict["num_nodes"] = dataset.num_nodes
hyperparams = Parameters(**hyperparams_dict)

In [0]:
trainer = Trainer(hyperparams=hyperparams, logdir=LOGDIR)
trainer.fit(dataset=dataset)

Fitting model 1 out of 5, repeat=0;epochs=60;steps_per_epoch=800;num_stacks=3
Epoch 1/60


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


800/800 [==============================] - 30s 38ms/step - loss: 4.9114 - lr: 0.0010 - mae_val: 3.5758 - mae_test: 3.9717 - mae_val_h1: 2.7057 - mae_test_h1: 2.9661 - mape_val_h1: 7.8483 - rmse_val_h1: 4.8951 - mape_test_h1: 8.2848 - rmse_test_h1: 5.1293 - mae_val_h2: 2.9168 - mae_test_h2: 3.2434 - mape_val_h2: 8.5222 - rmse_val_h2: 5.4642 - mape_test_h2: 9.0817 - rmse_test_h2: 5.7680 - mae_val_h3: 3.0999 - mae_test_h3: 3.3907 - mape_val_h3: 9.4375 - rmse_val_h3: 5.9925 - mape_test_h3: 9.9787 - rmse_test_h3: 6.2986 - mae_val_h4: 3.2438 - mae_test_h4: 3.6253 - mape_val_h4: 9.8125 - rmse_val_h4: 6.3329 - mape_test_h4: 10.4191 - rmse_test_h4: 6.6845 - mae_val_h5: 3.5651 - mae_test_h5: 3.9866 - mape_val_h5: 10.9355 - rmse_val_h5: 6.7539 - mape_test_h5: 11.5365 - rmse_test_h5: 7.1198 - mae_val_h6: 3.5622 - mae_test_h6: 3.8952 - mape_val_h6: 11.3959 - rmse_val_h6: 7.0862 - mape_test_h6: 11.8112 - rmse_test_h6: 7.4116 - mae_val_h7: 3.7587 - mae_test_h7: 4.1973 - mape_val_h7: 11.8924 - rmse_va

In [0]:
node_to_plot = 101
model = 0

node_id = trainer.models[model].fcgaga_layers[0].node_id_em(np.arange(207))
node_embeddings_dp = np.matmul(node_id, np.transpose(node_id, [1,0]))
node_embeddings_dp = tf.math.exp(10*node_embeddings_dp) 
plt.plot(node_embeddings_dp[node_to_plot])

node_id2 = trainer.models[model].fcgaga_layers[1].node_id_em(np.arange(207))
node_embeddings_dp2 = np.matmul(node_id2, np.transpose(node_id2, [1,0]))
node_embeddings_dp2 = tf.math.exp(10*node_embeddings_dp2) 
plt.plot(node_embeddings_dp2[node_to_plot])

node_id3 = trainer.models[model].fcgaga_layers[2].node_id_em(np.arange(207))
node_embeddings_dp3 = np.matmul(node_id3, np.transpose(node_id3, [1,0]))
node_embeddings_dp3 = tf.math.exp(10*node_embeddings_dp3) 
plt.plot(node_embeddings_dp3[node_to_plot])

In [0]:
import fnmatch

filt = "*"

plt.figure(figsize=(20,5))
legend = []
for i, h in enumerate(trainer.history):
    if fnmatch.fnmatch(trainer.folder_names[i], filt):
        plt.plot(h['mae_test'][10:])
        legend.append(trainer.folder_names[i])

plt.legend(legend)
plt.grid(True)
plt.ylim([3.0, 3.5])


In [0]:
def insert_dict(d, k, v):
    previous = d.get(k, [])
    d[k] = previous + [v]
    return d

early_stop_mae_h_repeats = dict()
early_stop_mape_h_repeats = dict()
early_stop_rmse_h_repeats = dict()
early_stop_mae_h_ave = dict()
early_stop_mape_h_ave = dict()
early_stop_rmse_h_ave = dict()
for i, h in enumerate(trainer.history):
    early_stop_idx = np.argmin(h['mae_val'])
    early_stop_mae = np.round(h['mae_test'][early_stop_idx], decimals=3)
    print(f"Early stop test error model {trainer.folder_names[i]}:", "Avg MAE", early_stop_mae)
    for horizon in range(3, hyperparams.horizon+1, 3):

        early_stop_mae_h_repeats = insert_dict(early_stop_mae_h_repeats, k=horizon, v=h[f'mae_test_h{horizon}'][early_stop_idx])
        early_stop_mape_h_repeats = insert_dict(early_stop_mape_h_repeats, k=horizon, v=h[f'mape_test_h{horizon}'][early_stop_idx])
        early_stop_rmse_h_repeats = insert_dict(early_stop_rmse_h_repeats, k=horizon, v=h[f'rmse_test_h{horizon}'][early_stop_idx])
        
        print(f"Horizon {horizon} MAE:", np.round(early_stop_mae_h_repeats[horizon][-1], decimals=2), 
              f"Horizon {horizon} MAPE:", np.round(early_stop_mape_h_repeats[horizon][-1], decimals=2), 
              f"Horizon {horizon} RMSE:", np.round(early_stop_rmse_h_repeats[horizon][-1], decimals=2))

    for horizon in range(3, hyperparams.horizon+1, 3):
        early_stop_mae_h_ave[horizon] = np.round(np.mean(early_stop_mae_h_repeats[horizon]), decimals=2)
        early_stop_mape_h_ave[horizon] = np.round(np.mean(early_stop_mape_h_repeats[horizon]), decimals=2)
        early_stop_rmse_h_ave[horizon] = np.round(np.mean(early_stop_rmse_h_repeats[horizon]), decimals=2)

print()
print("Average MAE:", early_stop_mae_h_ave)
print("Average MAPE:", early_stop_mape_h_ave)
print("Average RMSE:", early_stop_rmse_h_ave)
